In [1]:
# SRC comes from this link: https://fenicsproject.org/pub/tutorial/html/._ftut1008.html
# Version 1.0.0
# 11/23/2021: version 1.0.0. The code can simulate deformation subjuected to far-fielding shear loading. (D.Liu)

# Elastic deformation from 3D faulting simulated by modulus reduction in the fault zone.
# Created on 11/23/2021 by Dunyu Liu (dliu@ig.utexas.edu)

In [2]:
"""
Linear elastic problem.

  -div(sigma(u)) = f

The model is used to simulate earthquake deformation. 
Faulting is simulated via modulus reduction in the fault zone. 
"""

from __future__ import print_function
from fenics import *
from ufl import nabla_div

# Model domain.
L = 5e3; W = 5e3; Depth = 5e3
n1, n2, n3 = 10, 10, 10

# Rock properties
vp = 6000
vs = 3464
rho = 2670
mu = vs*vs*rho
# shear_modulus inside the fault zone
mu_fault = mu*0.5

lambda_ = vp*vp*rho - 2*mu
g = 0 # gravitiy, assumed to be zero in this application.


shear_slip = 100 # far-field loading by slip
fault_zone_width = 1e3 # fault zone width

dim = 3

# Create mesh and define function space
mesh = BoxMesh(Point(0, 0, 0), Point(L, W, Depth), n1, n2, n3)
V = VectorFunctionSpace(mesh, 'P', 1)
boundaries = MeshFunction("size_t", mesh, dim-1)

tol = 1E-14

# Define boundary condition
class Top_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[2], Depth, DOLFIN_EPS) and on_boundary
class Bottom_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[2], 0.0, DOLFIN_EPS) and on_boundary
class Left_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 0, DOLFIN_EPS) and on_boundary
class Right_boundary(SubDomain):
    def inside(self, x, on_boundary):   
        return near(x[0], L, DOLFIN_EPS) and on_boundary
class North_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], W, DOLFIN_EPS) and on_boundary
class South_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 0, DOLFIN_EPS) and on_boundary    
boundaries.set_all(0)
Top_boundary().mark(boundaries, 1)
Bottom_boundary().mark(boundaries, 2)
Left_boundary().mark(boundaries, 3)
Right_boundary().mark(boundaries, 4)
North_boundary().mark(boundaries, 5)
South_boundary().mark(boundaries, 6)
# Rename boundaries
top = 1
bottom = 2
left = 3
right = 4
north = 5
south = 6
# setting boundary conditions
bc = [DirichletBC(V.sub(2), Constant((0.0)), boundaries, bottom),
        DirichletBC(V.sub(0), Constant((0.0)), boundaries, left),
        DirichletBC(V.sub(0), Constant((0.0)), boundaries, right),
        DirichletBC(V.sub(1), shear_slip, boundaries, left), # far-field loading via slip
        DirichletBC(V.sub(1), -shear_slip, boundaries, right)] # far-field loading via slip


# Define fault zone as subdomains.
# At the center of x axis.
class Omega_0(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] >= L/2-fault_zone_width/2 and x[1] <= L/2+fault_zone_width/2
mf = MeshFunction("size_t", mesh, 3)
subdomain0 = Omega_0()
#subdomain1 = Omega_1()
mf.set_all(10)
subdomain0.mark(mf,11) # 1, the middle anisotrpic layer.
rock = 10
fault = 11

# Define strain and stress
def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)
    #return sym(nabla_grad(u))

def sigma(u):
    return lambda_*nabla_div(u)*Identity(d) + 2*shear_mod*epsilon(u)

# Define class to set different parameters to different subdomains
class K(UserExpression):
            def __init__(self, subdomains, k_fault, k_rock, **kwargs):
                super().__init__(**kwargs)
                self.subdomains = subdomains
                self.k_fault = k_fault
                self.k_rock = k_rock

            def eval_cell(self, values, x, cell):
                if self.subdomains[cell.index] == fault: #anisotrpic layer
                    values[0] = self.k_fault
                elif self.subdomains[cell.index] == rock:
                    values[0] = self.k_rock
            def values_shape(self):
                return (1,)            
shear_mod = K(mf, mu_fault, mu, degree=0)

# Define variational problem
u = TrialFunction(V)
d = u.geometric_dimension()  # space dimension
v = TestFunction(V)
f = Constant((0, 0, -rho*g))
T = Constant((0, 0, 0))
a = inner(sigma(u), epsilon(v))*dx
L = dot(f, v)*dx + dot(T, v)*ds

# Compute solution
u = Function(V)
solve(a == L, u, bc)

# Plot solution
#plot(u, title='Displacement', mode='displacement')

# Plot stress
s = sigma(u) - (1./3)*tr(sigma(u))*Identity(d)  # deviatoric stress
von_Mises = sqrt(3./2*inner(s, s))
V = FunctionSpace(mesh, 'P', 1)
von_Mises = project(von_Mises, V)
#plot(von_Mises, title='Stress intensity')

# Compute magnitude of displacement
u_magnitude = sqrt(dot(u, u))
u_magnitude = project(u_magnitude, V)
#plot(u_magnitude, 'Displacement magnitude')
#print('min/max u:',
#      u_magnitude.vector().array().min(),
#      u_magnitude.vector().array().max())

# Save solution to file in VTK format
File('elasticity/displacement.pvd') << u
#File('elasticity/von_mises.pvd') << von_Mises
#File('elasticity/magnitude.pvd') << u_magnitude

# Hold plot
#interactive()
print('Simulation Done')

Simulation Done
